Intentemos conseguir los mismo con transformadores customizados

In [7]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_classification
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report
from sklearn.compose import ColumnTransformer
df = pd.DataFrame({"col1":[-0.73,-2.36, 2.17, np.nan, -0.82, 1.83,  1.66, np.nan,  0.78 ,  1.56],
                   "col2":[ 0.21, -0.089 , -1.34,  2.57,  1.81, -1.77, -0.28, -0.73, -0.99, -1.54],
                   "col3":[-0.92, np.nan,  1.92 , -1.42 , -0.38, 1.47,  np.nan, -1.79,  np.nan,  1.22],
                   "cat":["B", "B", "M", "B", "B", "M", np.nan, "B", "M", "M"],
                   "target":[0, 0, 1, 0, 0, 1, 1, 0, 1, 1]})
#sns.pairplot(df[["col1", "col2", "col3", "target"]], hue='target')
X = df.drop("target", axis=1)
y = df["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3, test_size=0.3)
class ColumnImputer(BaseEstimator, TransformerMixin):
    def __init__(self, imputer=SimpleImputer(strategy="mean"), columns=None):
        self.imputer = imputer
        self.columns = columns

    def fit(self, X, y=None):
        self.imputer.fit(X[self.columns])
        return self    
    def get_feature_names_out(self):
        return self.imputer.get_feature_names_out()
    def transform(self, X):
        Xc = X.copy()
        Xc.loc[:, self.columns] = self.imputer.transform(X[self.columns])
        return Xc
    
class ColumnOneHot(BaseEstimator, TransformerMixin):
    def __init__(self, encoder=OneHotEncoder(drop='first', sparse_output=False), columns=None):
        self.encoder = encoder
        self.columns = columns

    def fit(self, X, y=None):
        self.encoder.fit(X[self.columns])
        return self    
    def get_feature_names_out(self):
        return self.encoder.get_feature_names_out()
    
    def transform(self, X):
        Xc = X.copy()
        Xc.loc[:, self.encoder.get_feature_names_out()] = self.encoder.transform(Xc[self.columns])
        Xc.drop(columns=self.columns, inplace=True)
        return Xc

In [12]:
pl = Pipeline(steps=[("imp_col1", ColumnImputer(imputer=SimpleImputer(strategy="mean"), columns=["col1"])),
                     ("imp_col3", ColumnImputer(imputer=SimpleImputer(strategy="median"), columns=["col3"])),
                     ("imp_cat", ColumnImputer(imputer=SimpleImputer(strategy="most_frequent"), columns=["cat"])),
                     ("onehot", ColumnOneHot(encoder=OneHotEncoder(drop='first', sparse_output=False, dtype=int), columns=["cat"])),
                     ("knn", KNeighborsClassifier(n_neighbors=3))])
                     
pl[:-1].fit_transform(X_train)

,col1,col2,col3,cat_M
2,2.170,-1.34,1.92,1
9,1.560,-1.54,1.22,1
6,1.660,-0.28,-0.92,0
7,1.088,-0.73,-1.79,0
0,-0.730,0.21,-0.92,0
3,1.088,2.57,-1.42,0
8,0.780,-0.99,-0.92,1


In [15]:
pl.fit(X_train, y_train)
y_pred = pl.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         1

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3

